### Form990 Merging

In [2]:
import pandas as pd
import polars as pl


In [22]:
df_20 = pl.read_excel('../../data/Form990/20form990.xlsx')

KeyboardInterrupt: 

In [20]:
excel_files_with_year = [
    ("../../data/Form990/20form990.xlsx", 2020),
    ("../../data/Form990/21form990.xlsx", 2021),
    ("../../data/Form990/22form990.xlsx", 2022)
]
unioned_form990 = pl.concat([
    pl.read_excel(file).with_columns(pl.lit(year).alias("year"))
    for file, year in excel_files_with_year
])

XlsxValueError: Error: potential invalid date format.

In [ ]:
unioned_form990.to_csv('../../data/form990_20_22_merged.csv', index=False)


## Joining EIN and other related fields into form 990 data


In [25]:
embf_merged.head()

EIN,NAME,ICO,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,DEDUCTIBILITY,FOUNDATION,ACTIVITY,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,SORT_NAME
i64,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,f64,f64,f64,str,str
4101,"""SOUTH LAFOURCHE QUARTERBACK CL…",null,"""167 BENT CYPRESS LN""","""LOCKPORT""","""LA""","""70374-4284""",0,3,3,1000,202005,1,4,0,1,1,null,0,0,0,1,12,null,null,null,"""N65""",null
260049,"""CORINTH BAPTIST CHURCH""",null,"""PO BOX 92""","""HOSFORD""","""FL""","""32334-0092""",2024,3,9,7200,196908,1,10,59029000,5,1,null,0,0,6,0,12,null,null,null,null,null
360268,"""IGLESIA VICTORIA""",null,"""1445 N BOONVILLE AVE""","""SPRINGFIELD""","""MO""","""65802-1894""",1678,3,9,7000,196408,1,10,1002029,5,1,null,0,0,6,0,12,null,null,null,null,null
490336,"""EASTSIDE BAPTIST CHURCH""",null,"""PO BOX 296""","""LABELLE""","""FL""","""33975-0296""",2024,3,9,7200,196908,1,10,59029000,5,1,null,0,0,6,0,12,null,null,null,null,null
2296179,"""RELIGIOUS SCIENCE CHURCH CENTE…",null,"""4102 MARLBOROUGH""","""SAN DIEGO""","""CA""","""92105-1462""",0,3,3,7000,196204,1,10,1000000,1,1,null,0,0,6,0,3,null,null,null,null,null


In [27]:
print(unioned_form990.shape)
print(embf_merged.shape)

(273971, 246)
(1870199, 28)


In [28]:
unioned_form990 = pd.read_csv('../../data/Form990/form990_20_22_merged.csv')


/var/folders/79/jdv69xcs22z7cyxln5rsbzzh0000gn/T/ipykernel_36868/545258913.py:1: DtypeWarning: Columns (31,97,99,101,103,105,135,137,139,206,214) have mixed types. Specify dtype option on import or set low_memory=False.
  unioned_form990 = pd.read_csv('../../data/Form990/form990_20_22_merged.csv')


In [31]:
embf_merged = pd.read_csv('../../data/embf_merged.csv')

# Print the shapes of the dataframes
print(unioned_form990.shape)
print(embf_merged.shape)

(943010, 247)
(1870199, 28)


In [32]:
unioned_form990['year'].value_counts()


year
2021    342920
2022    326119
2020    273971
Name: count, dtype: int64

In [33]:
selected_columns = ['EIN', 'NTEE_CD', 'CITY', 'STATE', 'ZIP']
embf_filtered = embf_merged[selected_columns]


embf_filtered = embf_filtered.rename(columns={
    'EIN': 'ein',
})

result_df = unioned_form990.merge(
    embf_filtered,
    on='ein',
    how='left'
)

result_df.shape 

(943010, 251)

In [39]:
# how many orgs across the last 3 years are env? 
result_df['NTEE_CD'] = result_df['NTEE_CD'].fillna('')

filtered_df = result_df[result_df['NTEE_CD'].str.startswith('C')]
print(filtered_df.shape)
print(filtered_df.shape)




(943010, 247)
(16849, 251)


In [40]:
embf_merged['NTEE_CD'].head(20)

0      N65
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6     X21Z
7      NaN
8      NaN
9     O200
10     NaN
11     Y50
12     NaN
13     NaN
14     NaN
15     S81
16     NaN
17     NaN
18     NaN
19     NaN
Name: NTEE_CD, dtype: object

In [10]:
filtered_df = embf_merged[embf_merged['ACTIVITY'] == 0]
print(filtered_df.shape[0]/embf_merged.shape[0])


0.5999163725357569


In [43]:
result_df[result_df['NTEE_CD'].str.startswith('C')]

,efile,ein,tax_pd,subseccd,s501c3or4947a1cd,schdbind,politicalactvtscd,lbbyingactvtscd,subjto6033cd,dnradvisedfundscd,...,unreltxincls511tx509,subtotsuppinc509,netincunrelatd509,othrinc509,totsupp509,year,NTEE_CD,CITY,STATE,ZIP
264,P,830514816,201812,3,Y,Y,N,N,N,N,...,0,56,0,0,1361741.0,2020,C32,SAPELLO,NM,87745-5200
283,P,460758408,201812,3,Y,N,N,N,N,N,...,0,0,0,0,0.0,2020,C50,DAVIE,FL,33325-0000
313,P,251752974,201812,3,Y,Y,N,N,N,N,...,0,0,0,0,0.0,2020,C300,BERLIN,PA,15530-5513
335,P,611198452,201812,3,Y,Y,N,N,N,N,...,0,487098,50059,0,6482796.0,2020,C32Z,BOWLING GREEN,KY,42101-4041
378,P,465642613,201812,3,Y,Y,N,N,N,N,...,0,0,0,0,0.0,2020,C60,HAWARDEN,IA,51023-7510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942904,E,990522409,202012,3,Y,Y,N,N,N,N,...,0,0,0,0,0.0,2022,C30,HANA,HI,96713-0454
942907,E,996000894,202109,3,Y,Y,N,N,N,N,...,0,0,0,0,0.0,2022,C60Z,HNP,HI,96718-0000
942913,E,996001533,202106,3,Y,N,N,N,N,N,...,0,13711,0,2000,542118.0,2022,C420,HONOLULU,HI,96828-0840
942933,E,996006829,202012,3,Y,Y,N,N,N,N,...,0,492,0,0,446502.0,2022,C300,HONOLULU,HI,96813-4709


In [41]:
# Read out the file
result_df.to_csv('../../data/form990_embf.csv')

In [47]:
result_df.shape

(943010, 251)

In [50]:
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C0')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C1')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C3')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C5O')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C50')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C00')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C22')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C28')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C31')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C38')]
print(df_filtered.shape)
df_filtered = result_df[result_df['NTEE_CD'].str.startswith('C43')]
print(df_filtered.shape)

(2193, 251)
(902, 251)
(8502, 251)
(2, 251)
(803, 251)
(23, 251)
(12, 251)
(3, 251)
(3, 251)
(4, 251)
(2, 251)
